In [ ]:
from agno.agent import Agent
from agno.document.base import Document
from agno.knowledge.document import DocumentKnowledgeBase
from agno.vectordb.pgvector import PgVector
from agno.embedder.ollama import OllamaEmbedder



fun_facts = """
- Earth is the third planet from the Sun and the only known astronomical object to support life.
- Approximately 71% of Earth's surface is covered by water, with the Pacific Ocean being the largest.
- The Earth's atmosphere is composed mainly of nitrogen (78%) and oxygen (21%), with traces of other gases.
- Earth rotates on its axis once every 24 hours, leading to the cycle of day and night.
- The planet has one natural satellite, the Moon, which influences tides and stabilizes Earth's axial tilt.
- Earth's tectonic plates are constantly shifting, leading to geological activities like earthquakes and volcanic eruptions.
- The highest point on Earth is Mount Everest, standing at 8,848 meters (29,029 feet) above sea level.
- The deepest part of the ocean is the Mariana Trench, reaching depths of over 11,000 meters (36,000 feet).
- Earth has a diverse range of ecosystems, from rainforests and deserts to coral reefs and tundras.
- The planet's magnetic field protects life by deflecting harmful solar radiation and cosmic rays.
"""

embedder=OllamaEmbedder(id="nomic-embed-text:latest", dimensions=768) # 3072

# Load documents from the data/docs directory
documents = [Document(content=fun_facts)]

# Database connection URL
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
vector_db = PgVector(
        table_name="new-documents",
        db_url=db_url,
        embedder=embedder,
    )

# Create a knowledge base with the loaded documents
knowledge_base = DocumentKnowledgeBase(
    documents=documents,
    vector_db=vector_db,
    
)

# Load the knowledge base
knowledge_base.load(recreate=False)

# Create an agent with the knowledge base
agent = Agent(
    knowledge=knowledge_base,
)




INFO Loading knowledge base

INFO Added 0 documents to knowledge base

In [2]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from typing import List
from agno.document import Document



In [5]:
def youtube_parser(video_url: str) -> List[Document]:
    yt_vid_id = video_url.split("v=")[1]
    transcript_list = YouTubeTranscriptApi.list_transcripts(yt_vid_id)
    subtitle = None

    for transcript in transcript_list:
        if transcript.language_code == "en":
            subtitle = transcript.fetch()
            break
        else:
            subtitle = transcript.translate('en').fetch()
            break

    if subtitle is None:
        raise ValueError("No subtitles found or translation failed.")

    transcript_text = "\n".join([sub['text'] for sub in subtitle])

    # Create a Document object
    doc = Document(
        content=transcript_text,
    )

    return [doc]


In [6]:
youtube_parser("https://www.youtube.com/watch?v=GTrZAitjmiI")

[Document(content="in this video I'm going to show you how\nto completely automate your invoice\nprocessing with a human in the loop\napproach meaning that you will have the\nability to interact with this automation\nright from your phone this will be a\nstep-by-step process meaning once this\nautomation processes One Step it's going\nto wait for your response to make sure\nit understands your decision before\nmoving on to the next step and\nprocessing the next step of this\nautomation so I'm going to go ahead and\nstart with a demo and then I'm going to\ncome back and walk through step by step\nand show you what these different nodes\nare and how this thing will work if you\nwere to to import this on your own\nworkflow all right so right now on the\nleft hand side you can see this is my\ntelegram account again I'm using\ntelegram but you can use WhatsApp or\nyour phone as well if you want to build\nthose automations yourself uh but on the\nright hand side this is my workflow you\ncan 

In [14]:
import sys
import ollama
from agno.embedder.ollama import OllamaEmbedder

# def check_embedding_dimensions():
print("Checking embedding dimensions...")

# Direct Ollama API check
print("Using direct Ollama API:")
try:
    response = ollama.embeddings(model="nomic-embed-text:latest", prompt="This is a test sentence")
    embedding_resp = response.get('embedding', [])
    print(f"Actual dimensions from direct Ollama API: {len(embedding_resp)}")
except Exception as e:
    print(f"Error with direct Ollama API: {e}")

# Agno OllamaEmbedder check
print("\nUsing Agno OllamaEmbedder:")
try:
    embedder = OllamaEmbedder(id="nomic-embed-text:latest", dimensions=768)
    text = "This is a test sentence"
    embedding = embedder.get_embedding(text)
    print(f"Expected dimensions in config: 768")
    print(f"Actual dimensions from Agno OllamaEmbedder: {len(embedding)}")
except Exception as e:
    print(f"Error with Agno OllamaEmbedder: {e}")
    
#     return

# if __name__ == "__main__":
#     check_embedding_dimensions()

Checking embedding dimensions...
Using direct Ollama API:
Actual dimensions from direct Ollama API: 768

Using Agno OllamaEmbedder:
Expected dimensions in config: 768
Actual dimensions from Agno OllamaEmbedder: 768


In [17]:
import sqlalchemy
from sqlalchemy import text

# Use the same database connection string from your application
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

# Create engine
engine = sqlalchemy.create_engine(db_url)

# Drop the vector table
with engine.connect() as connection:
    # Start a transaction
    with connection.begin():
        # Drop the table
        connection.execute(text("DROP TABLE IF EXISTS ai.agentic_rag_documents"))
        print("Table ai.agentic_rag_documents dropped successfully.")
        
        # Optional: If you need to recreate the schema too
        # connection.execute(text("DROP SCHEMA IF EXISTS ai CASCADE"))
        # connection.execute(text("CREATE SCHEMA ai"))
        # print("Schema ai recreated.")

print("Database reset complete. The table will be recreated with correct dimensions on next application run.")

Table ai.agentic_rag_documents dropped successfully.
Database reset complete. The table will be recreated with correct dimensions on next application run.


In [22]:
import sqlalchemy
from sqlalchemy import text, inspect

# Use the same database connection string
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
engine = sqlalchemy.create_engine(db_url)

# Method 1: Check using PostgreSQL information_schema
with engine.connect() as connection:
    result = connection.execute(text(
        "SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_schema = 'ai' AND table_name = 'agentic_rag_documents')"
    ))
    table_exists = result.scalar()
    print(f"Table exists (information_schema check): {table_exists}")

# Method 2: Try to select from the table (will raise an error if it doesn't exist)
try:
    with engine.connect() as connection:
        connection.execute(text("SELECT COUNT(*) FROM ai.agentic_rag_documents LIMIT 1"))
        print("Table still exists! The deletion may have failed.")
except Exception as e:
    print(f"Confirmed table doesn't exist: {e}")

# Method 3: List all tables in the schema
with engine.connect() as connection:
    result = connection.execute(text(
        "SELECT table_name FROM information_schema.tables WHERE table_schema = 'ai'"
    ))
    tables = [row[0] for row in result]
    print(f"Tables in 'ai' schema: {tables}")
    print(f"Is 'agentic_rag_documents' in list of tables: {'agentic_rag_documents' in tables}")

Table exists (information_schema check): False
Confirmed table doesn't exist: (psycopg.errors.UndefinedTable) relation "ai.agentic_rag_documents" does not exist
LINE 1: SELECT COUNT(*) FROM ai.agentic_rag_documents LIMIT 1
                             ^
[SQL: SELECT COUNT(*) FROM ai.agentic_rag_documents LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/f405)
Tables in 'ai' schema: ['agentic_rag_agent_sessions', 'agent_memory']
Is 'agentic_rag_documents' in list of tables: False


In [41]:
import sqlalchemy
from sqlalchemy import text

# Use the same database connection string
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
engine = sqlalchemy.create_engine(db_url)

# First get all tables in the schema
with engine.connect() as connection:
    result = connection.execute(text(
        "SELECT table_name FROM information_schema.tables WHERE table_schema = 'ai'"
    ))
    tables = [row[0] for row in result]
    
    print(f"Found {len(tables)} tables in 'ai' schema: {tables}")
    


Found 3 tables in 'ai' schema: ['agentic_rag_agent_sessions', 'agent_memory', 'agentic_rag_documents']


In [40]:
# Drop all tables
with engine.connect() as connection:
    for table in tables:
        print(f"Dropping table ai.{table}...")
        connection.execute(text(f"DROP TABLE IF EXISTS ai.{table}"))
            
    print("\nAll tables dropped successfully.")
        
        # Optional: Recreate schema
        # connection.execute(text("DROP SCHEMA IF EXISTS ai CASCADE"))
        # connection.execute(text("CREATE SCHEMA ai"))
        # print("Schema ai recreated from scratch.")

print("Database reset complete. All tables will be recreated with correct dimensions on next application run.")

IndentationError: expected an indented block after 'with' statement on line 2 (4115963647.py, line 3)

In [45]:
with engine.connect() as connection:
    result = connection.execute(text(
            "SELECT * FROM agent_memory"

        ))
    # result = connection.execute(text(f"DROP TABLE IF EXISTS agentic_rag_agent_sessions"))
result.fetchall()

[]

In [2]:
# Add this to your notebook to completely reset the database
import sqlalchemy
from sqlalchemy import text

# Use the same database connection string
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
engine = sqlalchemy.create_engine(db_url)

# Drop and recreate the schema
with engine.connect() as connection:
    with connection.begin():
        # Drop entire schema with CASCADE to remove all objects
        connection.execute(text("DROP SCHEMA IF EXISTS ai CASCADE"))
        
        # Recreate the schema 
        connection.execute(text("CREATE SCHEMA ai"))
        
        print("Schema completely reset. The application will create all tables with the correct schema on next run.")
        

Schema completely reset. The application will create all tables with the correct schema on next run.


In [6]:
# Check if Agno has an upgrade utility
try:
    from agno.db.upgrade import upgrade_schema
    
    # Run the schema upgrade
    upgrade_schema(db_url="postgresql+psycopg://ai:ai@localhost:5532/ai")
    print("Schema upgrade completed successfully.")
except ImportError:
    print("Agno schema upgrade utility not found. Use the documentation link instead.")

Agno schema upgrade utility not found. Use the documentation link instead.


In [14]:
dir(model)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializ

In [43]:
from agno.tools.chat_history import get_chat_history

ModuleNotFoundError: No module named 'agno.tools.chat_history'